# Generate Images
March 1, 2021

In [1]:
import argparse
import os
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml
import collections

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
from utils import *

## Build Model structure

In [4]:
def f_load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    return config

def f_manual_add_argparse():
    ''' use only in jpt notebook'''
    args=argparse.Namespace()
    args.config='1_main_code/config_3d_cgan_128_cori.yaml'
    args.mode='fresh'
    args.ip_fldr=''
#     args.local_rank=0
    args.facility='cori'
    args.distributed=False
    args.ngpu=1
#     args.mode='continue'
#     args.ip_fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201211_093818_nb_test/'
    
    return args
             
    
def weights_init(m):
    '''custom weights initialization called on netG and netD '''
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def f_init_gdict(args,gdict):
    ''' Create global dictionary gdict from args and config file'''
    
    ## read config file
    config_file=args.config
    with open(config_file) as f:
        config_dict= yaml.load(f, Loader=yaml.SafeLoader)
        
    gdict=config_dict['parameters']

    args_dict=vars(args)
    ## Add args variables to gdict
    for key in args_dict.keys():
        gdict[key]=args_dict[key]
    return gdict

In [5]:
def f_gen_images(gdict,netG,optimizerG,sigma,ip_fname,op_loc,op_strg='inf_img_',op_size=500):
    '''Generate images for best saved models
     Arguments: gdict, netG, optimizerG, 
                 sigma : sigma input value
                 ip_fname: name of input file
                op_strg: [string name for output file]
                op_size: Number of images to generate
    '''

    nz,device=gdict['nz'],gdict['device']

    try:
        if torch.cuda.is_available(): checkpoint=torch.load(ip_fname)
        else: checkpoint=torch.load(ip_fname,map_location=torch.device('cpu'))
    except Exception as e:
        print(e)
        print("skipping generation of images for ",ip_fname)
        return
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
    
    ## Load other stuff
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    # Generate batch of latent vectors
    noise = torch.randn(op_size, 1, 1, 1, nz, device=device)
    tnsr_cosm_params=(torch.ones(op_size,device=device)*sigma).view(op_size,1)

    # Generate fake image batch with G
    netG.eval() ## This is required before running inference
    with torch.no_grad(): ## This is important. fails without it for multi-gpu
        gen = netG(noise,tnsr_cosm_params)
        gen_images=gen.detach().cpu().numpy()
        print(gen_images.shape)

    op_fname='%s_label-%s_epoch-%s_step-%s.npy'%(op_strg,sigma,epoch,iters)
    np.save(op_loc+op_fname,gen_images)
    
    print("Image saved in ",op_fname)

In [6]:
# Mod for 3D
def f_get_model(model_name,gdict):
    ''' Module to define Generator and Discriminator'''
#     print("Model name",model_name)

    if model_name==2: #### Concatenate sigma input
        if gdict['image_size']==64:
            class Generator(nn.Module):
                def __init__(self, gdict):
                    super(Generator, self).__init__()

                    ## Define new variables from dict
                    keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                    ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                    self.main = nn.Sequential(
                        # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                        nn.Linear(nz+1,nc*ngf*8**3),# 262144
                        nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        View(shape=[-1,ngf*8,4,4,4]),
                        nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                        nn.BatchNorm3d(ngf*4,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        # state size. (ngf*4) x 8 x 8
                        nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                        nn.BatchNorm3d(ngf*2,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        # state size. (ngf*2) x 16 x 16
                        nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                        nn.BatchNorm3d(ngf,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        # state size. (ngf) x 32 x 32
                        nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                        nn.Tanh()
                    )

                def forward(self, noise,labels):
                    x=labels.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).float()
                    gen_input=torch.cat((noise,x),-1)
                    img=self.main(gen_input)

                    return img

            class Discriminator(nn.Module):
                def __init__(self, gdict):
                    super(Discriminator, self).__init__()

                    ## Define new variables from dict
                    keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                    ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                    self.linear_transf=nn.Linear(4,4)
                    self.main = nn.Sequential(
                        # input is (nc) x 64 x 64
                        # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                        nn.Conv3d(nc+1, ndf,kernel_size, stride, d_padding,  bias=True),
                        nn.BatchNorm3d(ndf,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf) x 32 x 32
                        nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                        nn.BatchNorm3d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf*2) x 16 x 16
                        nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                        nn.BatchNorm3d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf*4) x 8 x 8
                        nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                        nn.BatchNorm3d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf*8) x 4 x 4
                        nn.Flatten(),
                        nn.Linear(nc*ndf*8*8*8, 1)
            #             nn.Sigmoid()
                    )

                def forward(self, img,labels):
                    img_size=gdict['image_size']
                    x=labels.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).repeat(1,1,1,1,4).float() # get to size (batch,1,1,1,4)
                    x=self.linear_transf(x)
                    x=torch.repeat_interleave(x,int((img_size*img_size*img_size)/4)) # get to size (batch* img^3)
                    x=x.view(labels.size(0),1,img_size,img_size,img_size) ## Get to size (batch,1,img,img,img)

                    ip=torch.cat((img,x),axis=1)

                    results=[ip]
                    lst_idx=[]
                    for i,submodel in enumerate(self.main.children()):
                        mid_output=submodel(results[-1])
                        results.append(mid_output)
                        ## Select indices in list corresponding to output of Conv layers
                        if submodel.__class__.__name__.startswith('Conv'):
            #                 print(submodel.__class__.__name__)
            #                 print(mid_output.shape)
                            lst_idx.append(i)

                    FMloss=True
                    if FMloss:
                        ans=[results[1:][i] for i in lst_idx + [-1]]
                    else :
                        ans=results[-1]

                    return ans    
        
        elif gdict['image_size']==128:

            class Generator(nn.Module):
                def __init__(self, gdict):
                    super(Generator, self).__init__()

                    ## Define new variables from dict
                    keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                    ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                    self.main = nn.Sequential(
                        # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                        nn.Linear(nz+1,nc*ngf*8**3*8),# 262144
                        nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        View(shape=[-1,ngf*8,8,8,8]),
                        nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                        nn.BatchNorm3d(ngf*4,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        # state size. (ngf*4) x 8 x 8
                        nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                        nn.BatchNorm3d(ngf*2,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        # state size. (ngf*2) x 16 x 16
                        nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                        nn.BatchNorm3d(ngf,eps=1e-05, momentum=0.9, affine=True),
                        nn.ReLU(inplace=True),
                        # state size. (ngf) x 32 x 32
                        nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                        nn.Tanh()
                    )

                def forward(self, noise,labels):
                    x=labels.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).float()
                    gen_input=torch.cat((noise,x),-1)
                    img=self.main(gen_input)

                    return img
                

            class Discriminator(nn.Module):
                def __init__(self, gdict):
                    super(Discriminator, self).__init__()

                    ## Define new variables from dict
                    keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                    ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                    self.linear_transf=nn.Linear(4,4)
                    self.main = nn.Sequential(
                        # input is (nc) x 64 x 64
                        # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                        nn.Conv3d(nc+1, ndf,kernel_size, stride, d_padding,  bias=True),
                        nn.BatchNorm3d(ndf,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf) x 32 x 32
                        nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                        nn.BatchNorm3d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf*2) x 16 x 16
                        nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                        nn.BatchNorm3d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf*4) x 8 x 8
                        nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                        nn.BatchNorm3d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
                        nn.LeakyReLU(0.2, inplace=True),
                        # state size. (ndf*8) x 4 x 4
                        nn.Flatten(),
                        nn.Linear(nc*ndf*8*8*8*8, 1)
            #             nn.Sigmoid()
                    )

                def forward(self, img,labels):
                    img_size=gdict['image_size']
                    x=labels.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).repeat(1,1,1,1,4).float() # get to size (batch,1,1,1,4)
                    x=self.linear_transf(x)
                    x=torch.repeat_interleave(x,int((img_size*img_size*img_size)/4)) # get to size (batch* img^3)
                    x=x.view(labels.size(0),1,img_size,img_size,img_size) ## Get to size (batch,1,img,img,img)

                    ip=torch.cat((img,x),axis=1)

                    results=[ip]
                    lst_idx=[]
                    for i,submodel in enumerate(self.main.children()):
                        mid_output=submodel(results[-1])
                        results.append(mid_output)
                        ## Select indices in list corresponding to output of Conv layers
                        if submodel.__class__.__name__.startswith('Conv'):
            #                 print(submodel.__class__.__name__)
            #                 print(mid_output.shape)
                            lst_idx.append(i)

                    FMloss=True
                    if FMloss:
                        ans=[results[1:][i] for i in lst_idx + [-1]]
                    else :
                        ans=results[-1]

                    return ans 
                
                
                
    elif model_name==3:#### Model 3: with ConditionalInstanceNorm2d,
        class ConditionalInstanceNorm2d(nn.Module):
            def __init__(self, num_features, num_params):
                super().__init__()
                self.num_features = num_features
                self.InstNorm = nn.InstanceNorm2d(num_features, affine=False)
                self.affine = nn.Linear(num_params, num_features * 2)
                self.affine.weight.data[:, :num_features].normal_(1, 0.02)  # Initialise scale at N(1, 0.02)
                self.affine.weight.data[:, num_features:].zero_()  # Initialise bias at 0

            def forward(self, x, y):
                out = self.InstNorm(x)
                gamma, beta = self.affine(y).chunk(2, 1)
                out = gamma.view(-1, self.num_features, 1, 1) * out + beta.view(-1, self.num_features, 1, 1)
                return out

        class ConditionalSequential(nn.Sequential):
            def __init__(self,*args):
                super(ConditionalSequential, self).__init__(*args)

            def forward(self, inputs, labels):
                for module in self:
                    if module.__class__ is ConditionalInstanceNorm2d:
                        inputs = module(inputs, labels.float())
                    else:
                        inputs = module(inputs)

                return inputs

        class Generator(nn.Module):
            def __init__(self, gdict):
                super(Generator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                self.main = ConditionalSequential(
                    # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Linear(nz,nc*ngf*8**3),# 262144
                    nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    View(shape=[-1,ngf*8,4,4,4]),
                    nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                    ConditionalInstanceNorm2d(ngf*4,1),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*4) x 8 x 8
                    nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                    ConditionalInstanceNorm2d(ngf*2,1),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*2) x 16 x 16
                    nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                    ConditionalInstanceNorm2d(ngf,1),
                    nn.ReLU(inplace=True),
                    # state size. (ngf) x 32 x 32
                    nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                    nn.Tanh()
                )
                
                
            def forward(self, noise,labels):
                img=self.main(noise,labels)

                return img

        class Discriminator(nn.Module):
            def __init__(self, gdict):
                super(Discriminator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())
                
                self.main = nn.Sequential(
                    # input is (nc) x 64 x 64
                    # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Conv3d(nc, ndf,kernel_size, stride, d_padding,  bias=True),
                    ConditionalInstanceNorm2d(ndf,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf) x 32 x 32
                    nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                    ConditionalInstanceNorm2d(ndf*2,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*2) x 16 x 16
                    nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                    ConditionalInstanceNorm2d(ndf*4,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*4) x 8 x 8
                    nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                    ConditionalInstanceNorm2d(ndf*8,1),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*8) x 4 x 4
                    nn.Flatten(),
                    nn.Linear(nc*ndf*8*8*8, 1)
        #             nn.Sigmoid()
                )
        
            def forward(self, ip,labels):   
                results=[ip]
                lst_idx=[]
                for i,submodel in enumerate(self.main.children()):
                    mid_output=submodel(results[-1])
                    results.append(mid_output)
                    ## Select indices in list corresponding to output of Conv layers
                    if submodel.__class__.__name__.startswith('Conv'):
        #                 print(submodel.__class__.__name__)
        #                 print(mid_output.shape)
                        lst_idx.append(i)

                FMloss=True
                if FMloss:
                    ans=[results[1:][i] for i in lst_idx + [-1]]
                else :
                    ans=results[-1]
                return ans

    return Generator, Discriminator

In [7]:
if __name__=="__main__":
    torch.backends.cudnn.benchmark=True
    t0=time.time()
    #################################
    args=f_manual_add_argparse()
    ### Set up ###

    # Initilize variables    
    gdict={}
    gdict=f_init_gdict(args,gdict)
    print(gdict)
    ## Add args variables to gdict
#     for key in ['ngpu']:
#         gdict[key]=vars(args)[key]
    gdict['device']=torch.device("cuda" if (torch.cuda.is_available() and gdict['ngpu'] > 0) else "cpu")
    gdict['ngpu']=torch.cuda.device_count()
    gdict['multi-gpu']=True if (gdict['device'].type == 'cuda') and (gdict['ngpu'] > 1) else False 

    Generator, Discriminator=f_get_model(gdict['model'],gdict)
    print("Building GAN networks")
    # Create Generator
    netG = Generator(gdict).to(gdict['device'])
    netG.apply(weights_init)
    print(netG)
    # summary(netG,(1,1,64))
    
    print("Number of GPUs used %s"%(gdict['ngpu']))
    if (gdict['multi-gpu']):
        netG = nn.DataParallel(netG, list(range(gdict['ngpu'])))

    optimizerG = optim.Adam(netG.parameters(), lr=gdict['learn_rate_g'], betas=(gdict['beta1'], 0.999),eps=1e-7)



{'ip_fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube', 'op_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/', 'image_size': 128, 'num_imgs': 5000, 'ip_fldr': '', 'chkpt_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210722_200025_cgan_128_nodes1_lr0.00002_finetune/models/checkpoint_last.tar', 'workers': 2, 'nc': 1, 'nz': 64, 'ngf': 64, 'ndf': 64, 'beta1': 0.5, 'kernel_size': 5, 'stride': 2, 'g_padding': 2, 'd_padding': 2, 'flip_prob': 0.01, 'bns': 50, 'checkpoint_size': 10, 'sigma_list': [0.5, 0.8, 1.1], 'model': 2, 'batch_size': 8, 'epochs': 300, 'op_size': 8, 'learn_rate_d': 2e-06, 'learn_rate_g': 2e-06, 'lr_d_epochs': [40, 60], 'lr_d_gamma': 1.0, 'lr_g_epochs': [40, 60], 'lr_g_gamma': 1.0, 'deterministic': False, 'seed': 234373, 'lambda_spec_mean': 0.1, 'lambda_spec_var': 0.1, 'lambda_fm': 50.0, 'lambda

In [8]:
gdict

{'ip_fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube',
 'op_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
 'image_size': 128,
 'num_imgs': 5000,
 'ip_fldr': '',
 'chkpt_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210722_200025_cgan_128_nodes1_lr0.00002_finetune/models/checkpoint_last.tar',
 'workers': 2,
 'nc': 1,
 'nz': 64,
 'ngf': 64,
 'ndf': 64,
 'beta1': 0.5,
 'kernel_size': 5,
 'stride': 2,
 'g_padding': 2,
 'd_padding': 2,
 'flip_prob': 0.01,
 'bns': 50,
 'checkpoint_size': 10,
 'sigma_list': [0.5, 0.8, 1.1],
 'model': 2,
 'batch_size': 8,
 'epochs': 300,
 'op_size': 8,
 'learn_rate_d': 2e-06,
 'learn_rate_g': 2e-06,
 'lr_d_epochs': [40, 60],
 'lr_d_gamma': 1.0,
 'lr_g_epochs': [40, 60],
 'lr_g_gamma': 1.0,
 'deterministic': False,
 'seed': 234373,
 'lambda_spec_mean': 0.1,
 'lambda_spec_var'

## Run Inference

In [9]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/

20210615_72613_cgan_64_bs32_nodes8_lr0.0001_good_cgan_run/
20210617_204752_cgan-64_bs32_nodes1_lr0.0001-vary_fm50_goodrun/
20210619_224213_cgan_bs32_lr0.0001-vary_fm50_spec0.1/
20210620_113852_cgan_bs32_lr0.0001-vary_fm50_spec0.01/
20210620_63445_cgan_bs32_lr0.0001-vary_fm50_spec0.05/
20210626_100420_cgan-128_bs8_nodes1_lr0.0002_fm0_goodrun/
20210626_50127_cgan_bs32_lr0.0001-vary_fm50_spec0.5/
20210628_73858_cgan_128_nodes1_lr0.0002_fm0_goodrun/
20210703_50233_cgan_128_nodes1_lr0.0002_vary_fm0_goodrun/
20210707_65905_cgan_128_lr0.0002_vary_fm50_bad/
20210707_95421_cgan_128_bs8_nodes4_lr0.0004_gclip1.0_fm0_summit/
20210719_153427_cgan_128_bs8_nodes4_lr0.0004_gclip1.0_spec10/
20210719_153444_cgan_128_bs8_nodes4_lr0.0002_gclip1.0_fm0_spec50/
20210720_63509_cgan_128_bs8_nodes4_lr0.0004_gclip1.0_spec10_noadv/
20210720_81213_cgan_128_bs8_nodes4_lr0.0002_gclip1.0_spec50_noadv/
20210721_71301_cgan_128_bs8_nodes4_lr0.0004_gclip1.0_specfullsize/
20210721_71318_cgan_128_bs8_nodes4_lr0.0002_gclip1

In [9]:
# ## For single checkpoint
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# fldr='20210119_134802_cgan_predict_0.65_m2'
# param_label=1.1
# op_loc=main_dir+fldr+'/images/'
# ip_fname=main_dir+fldr+'/models/checkpoint_best_spec.tar'
# f_gen_images(gdict,netG,optimizerG,param_label,ip_fname,op_loc,op_strg='inference_spec',op_size=1000)


In [12]:
## For multiple checkpoints 
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'
fldr='20210726_173009_cgan_128_nodes1_lr0.000002_finetune'

param_list=[0.5,0.65,0.8,1.1]
op_loc=main_dir+fldr+'/images/'
step_list=[1740, 1950, 2580, 3010, 5590, 6800]

for step in step_list:
    for param_label in param_list:
        try:
    #     ip_fname=main_dir+fldr+'/models/checkpoint_{0}.tar'.format(step)
    #     f_gen_images(gdict,netG,optimizerG,param_label,ip_fname,op_loc,op_strg='inference_spec',op_size=1000)
            ip_fname=glob.glob(main_dir+fldr+'/models/checkpoint_*{0}.tar'.format(step))[0]
            print(ip_fname)
            f_gen_images(gdict,netG,optimizerG,param_label,ip_fname,op_loc,op_strg='inference',op_size=32)
        except Exception as e:
            print(e)
            print("skipping ",step) 

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210726_173009_cgan_128_nodes1_lr0.000002_finetune/models/checkpoint_1740.tar
(32, 1, 128, 128, 128)
Image saved in  inference_label-0.5_epoch-7_step-1740.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210726_173009_cgan_128_nodes1_lr0.000002_finetune/models/checkpoint_1740.tar
(32, 1, 128, 128, 128)
Image saved in  inference_label-0.65_epoch-7_step-1740.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210726_173009_cgan_128_nodes1_lr0.000002_finetune/models/checkpoint_1740.tar
(32, 1, 128, 128, 128)
Image saved in  inference_label-0.8_epoch-7_step-1740.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210726_173009_cgan_128_nodes1_lr0.000002_finetune/models/checkpoint_1740.tar
(32, 1, 128, 128, 128)
Image saved in  inference_label-1.1_epoch-7_s

In [ ]:
# fname=op_loc+'inference_spec_epoch-11_step-37040.npy'
# a1=np.load(fname)
# print(a1.shape)